<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [7]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [8]:
import folium
import pandas as pd

In [9]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [24]:
## Task 1: Mark all launch sites on a map
launch_sites = [
    ['CCAFS LC-40', 28.562302, -80.577356],
    ['CCAFS SLC-40', 28.563197, -80.576820],
    ['KSC LC-39A', 28.573255, -80.646895],
    ['VAFB SLC-4E', 34.632834, -120.610746]
]


launch_sites_df = pd.DataFrame(launch_sites, columns=['Launch Site', 'Lat', 'Long'])

import numpy as np
sites_df = (
    launch_sites_df if 'launch_sites_df' in globals() else
    launch_df        if 'launch_df' in globals() else
    df)

col_map = {
    'Launch Site':'LaunchSite',
    'Launch_Site':'LaunchSite',
    'Latitude':'Lat',
    'LAT':'Lat',
    'lat':'Lat',
    'Longitude':'Long',
    'LON':'Long',
    'lon':'Long'}
sites_df = sites_df.rename(columns={c: col_map.get(c, c) for c in sites_df.columns})

required = ['LaunchSite','Lat','Long']
missing = [c for c in required if c not in sites_df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}. Expected columns like {required}")

center = [sites_df['Lat'].astype(float).mean(), sites_df['Long'].astype(float).mean()]
m = folium.Map(location=center, zoom_start=5, tiles='CartoDB positron')

# Add a marker for each launch site
for _, r in sites_df.iterrows():
    folium.Marker(
        location=[float(r['Lat']), float(r['Long'])],
        popup=folium.Popup(str(r['LaunchSite']), max_width=250),
        tooltip=str(r['LaunchSite'])
    ).add_to(m)

m

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [14]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [15]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [16]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [17]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [27]:
df = launches_geo.copy()

# 1) Build a case-insensitive lookup for existing columns
lower_to_orig = {c.lower(): c for c in df.columns}

def pick_col(candidates):
    """Return the first existing column (original case) from a list of lowercase candidate names."""
    for cand in candidates:
        if cand in lower_to_orig:
            return lower_to_orig[cand]
    return None

# 2) Auto-detect columns
lat_col = pick_col(["lat", "latitude", "lat_x", "lat_y"])
lon_col = pick_col(["lon", "long", "longitude", "lng", "long_x", "long_y"])
site_col = pick_col(["launchsite", "site", "site_name", "name"])

# 3) Helpful error if we can’t find what we need
missing = []
if not lat_col: missing.append("latitude (e.g., Lat_x, Lat_y, Latitude, Lat)")
if not lon_col: missing.append("longitude (e.g., Long_x, Long_y, Longitude, Long/Lon)")
if not site_col: missing.append("site name (e.g., LaunchSite, Site, Site_Name, Name)")
if missing:
    raise KeyError(
        "Could not find required columns:\n- " + "\n- ".join(missing) +
        f"\n\nYour columns are: {list(df.columns)}"
    )

# 4) Standardize column names to 'lat', 'lon', 'site'
df = df.rename(columns={lat_col: "lat", lon_col: "lon", site_col: "site"}).copy()

# 5) Ensure coordinates are numeric and drop invalid rows
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
df = df.dropna(subset=["lat", "lon"])

# (Optional) De-duplicate sites by name/coordinate if your table has repeats
df = df.drop_duplicates(subset=["site", "lat", "lon"]).reset_index(drop=True)

# 6) Create the map centered on the mean coordinate
center = [df["lat"].mean(), df["lon"].mean()]
m = folium.Map(location=center, zoom_start=4, tiles="OpenStreetMap")

# 7) Add a Circle (plus a small marker) for each site
for _, r in df.iterrows():
    label = str(r["site"])
    folium.Circle(
        location=[r["lat"], r["lon"]],
        radius=5000,           # meters; tweak as needed
        color="blue",
        fill=True,
        fill_opacity=0.25,
        popup=folium.Popup(label, max_width=250),
    ).add_to(m)

    folium.Marker(
        location=[r["lat"], r["lon"]],
        tooltip=label
    ).add_to(m)

# 8) Display map in Jupyter (or save to HTML)
m

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [28]:
# Task 2: Mark the success/failed launches for each site on the map (MY STEPS)
def normalize_outcome(x):
    s = str(x).strip().lower()
    if "success" in s or "s" == s or "ok" in s or "pass" in s:
        return "Success"
    if "partial" in s:
        return "Partial"
    return "Failure"

df["Outcome"] = df["Class"].apply(normalize_outcome)

# Jitter so launches at the same site don’t perfectly overlap
rng = np.random.default_rng(42)
jitter_deg = 0.003  # ~300m

def jitter(v): 
    return float(v) + rng.normal(0, jitter_deg)

# Color mapping
def outcome_color(outcome):
    if outcome == "Success": return "green"
    if outcome == "Failure": return "red"
    return "orange"

# Add per-launch markers (CircleMarker) colored by outcome
for _, r in df.iterrows():
    folium.CircleMarker(
        location=[jitter(r["lat"]), jitter(r["lon"])],
        radius=5,
        color=outcome_color(r["Outcome"]),
        fill=True,
        fill_color=outcome_color(r["Outcome"]),
        fill_opacity=0.7,
        popup=folium.Popup(f"{r['site']} — {r['Class']}", max_width=260),
        tooltip=f"{r['site']} — {r['Outcome']}"
    ).add_to(m)

# Optional: simple legend
legend_html = """
<div style="
    position: fixed; bottom: 20px; left: 20px; z-index: 9999;
    background: white; padding: 10px 12px; border: 1px solid #ccc; border-radius: 8px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15); font-size: 14px;">
    <div style="font-weight: 600; margin-bottom: 6px;">Launch Outcomes</div>
    <div><span style="display:inline-block;width:10px;height:10px;background:green;border-radius:50%;margin-right:6px;"></span>Success</div>
    <div><span style="display:inline-block;width:10px;height:10px;background:red;border-radius:50%;margin-right:6px;"></span>Failure</div>
    <div><span style="display:inline-block;width:10px;height:10px;background:orange;border-radius:50%;margin-right:6px;"></span>Partial/Other</div>
</div>
"""
from branca.element import Element
m.get_root().html.add_child(Element(legend_html))

m

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [29]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [30]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [34]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df["Lat"]  = pd.to_numeric(spacex_df["Lat"], errors="coerce")
spacex_df["Long"] = pd.to_numeric(spacex_df["Long"], errors="coerce")
spacex_df = spacex_df.dropna(subset=["Lat", "Long", "class"])

# Create marker_color from 'class' (1=green, 0=red)
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

# Add per-launch markers (colored by outcome)
for _, row in spacex_df.iterrows():
    folium.CircleMarker(
        location=[row["Lat"], row["Long"]],
        radius=5,
        color=row["marker_color"],
        fill=True,
        fill_color=row["marker_color"],
        fill_opacity=0.7,
        popup=folium.Popup(f"{row['Launch Site']} — {'Success' if row['class'] == 1 else 'Failure'}", max_width=260),
        tooltip=f"{row['Launch Site']} — {'Success' if row['class'] == 1 else 'Failure'}",
    ).add_to(m)

m


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [35]:
if "marker_color" not in spacex_df.columns:
    spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")

# Add markers into the existing marker_cluster
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record["Lat"], record["Long"]],
        popup=folium.Popup(
            f"{record['Launch Site']} — {'Success' if record['class'] == 1 else 'Failure'}",
            max_width=260
        ),
        icon=folium.Icon(
            color="white",                   # pin body color
            icon_color=record["marker_color"],  # glyph color (green/red)
            icon="rocket",                  # nice FA icon; optional
            prefix="fa"                     # use Font Awesome
        ),
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [37]:
# TASK 3: Calculate the distances between a launch site to its proximities
proximities_df = pd.DataFrame([
    {"name":"HWY-528", "type":"Highway", "Lat":28.563, "Long":-80.577},
    {"name":"Rail-A",  "type":"Railway", "Lat":28.572, "Long":-80.585},
    {"name":"Cocoa",   "type":"City",    "Lat":28.386, "Long":-80.742},
    {"name":"Atlantic","type":"Coastline","Lat":28.400,"Long":-80.560},
])



import numpy as np

# Haversine distance helper (km)
def haversine_km(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 6371.0088 * 2 * np.arcsin(np.sqrt(a))

# Find nearest proximity of each type for each site
rows = []
for _, s in spacex_df.iterrows():
    for t in proximities_df["type"].unique():
        sub = proximities_df[proximities_df["type"] == t]
        dists = haversine_km(s["Lat"], s["Long"], sub["Lat"].values, sub["Long"].values)
        idx = int(np.argmin(dists))
        rows.append({
            "site": s["Launch Site"],
            "prox_type": t,
            "prox_name": sub.iloc[idx]["name"],
            "prox_lat": sub.iloc[idx]["Lat"],
            "prox_lon": sub.iloc[idx]["Long"],
            "distance_km": dists[idx]
        })

nearest_df = pd.DataFrame(rows)

# Draw lines from sites to nearest proximities
type_color = {"Highway":"red","Railway":"blue","City":"green","Coastline":"purple"}
for _, r in nearest_df.iterrows():
    folium.PolyLine(
        [(s["Lat"], s["Long"]), (r["prox_lat"], r["prox_lon"])],
        color=type_color.get(r["prox_type"], "orange"),
        weight=3,
        tooltip=f"{r['site']} → {r['prox_type']} ({r['distance_km']:.2f} km)"
    ).add_to(site_map)

    folium.CircleMarker(
        location=(r["prox_lat"], r["prox_lon"]),
        radius=4,
        color=type_color.get(r["prox_type"], "orange"),
        fill=True,
        fill_opacity=0.9,
        tooltip=f"{r['prox_type']}: {r['prox_name']} ({r['distance_km']:.2f} km)"
    ).add_to(site_map)

site_map

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [38]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [39]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [41]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    R = 6371.0088  # mean Earth radius (km)
    return R * c

# Example: one launch site and one coastline coordinate
launch_site_lat, launch_site_lon = 28.563, -80.577   # Kennedy Space Center
coastline_lat, coastline_lon     = 28.56367, -80.57163

distance_coastline = calculate_distance(
    launch_site_lat, launch_site_lon,coastline_lat, coastline_lon
)

print(f"Distance to coastline: {distance_coastline:.2f} km")


Distance to coastline: 0.53 km


In [42]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )



from folium.features import DivIcon

# Launch site coordinates (example: Kennedy Space Center)
launch_lat, launch_lon = 28.563, -80.577

# Closest coastline point (taken from MousePosition hover)
coast_lat, coast_lon = 28.56367, -80.57163

# Calculate distance
distance_km = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Add a marker at the coastline point
folium.Marker(
    location=[coast_lat, coast_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color="blue", icon="flag")
).add_to(site_map)

# Add a text marker showing the distance
distance_marker = folium.Marker(
    location=[coast_lat, coast_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
)
distance_marker.add_to(site_map)

# Optionally, draw a line between the site and coastline
folium.PolyLine(
    [(launch_lat, launch_lon), (coast_lat, coast_lon)],
    color="orange", weight=2.5, opacity=0.8
).add_to(site_map)

site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [43]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)


# Coordinates
launch_coords = [launch_lat, launch_lon]
coast_coords  = [coast_lat, coast_lon]

# Create PolyLine between site and coastline
lines = folium.PolyLine(
    locations=[launch_coords, coast_coords],
    weight=2,              # thickness of line
    color="orange",        # line color
    opacity=0.8
)

# Add to map
site_map.add_child(lines)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [44]:
import numpy as np
from folium.features import DivIcon

# Colors per proximity type (extend or tweak as needed)
type_color = {
    "Highway":   "#e41a1c",
    "Railway":   "#377eb8",
    "City":      "#4daf4a",
    "Coastline": "#984ea3",
}

# For each launch site, find the nearest of each proximity type and draw marker + line
for _, site_row in spacex_df.iterrows():
    s_name = site_row["Launch Site"]
    s_lat, s_lon = float(site_row["Lat"]), float(site_row["Long"])

    for prox_type in proximities_df["type"].unique():
        sub = proximities_df[proximities_df["type"] == prox_type]
        if sub.empty:
            continue

        # Compute distances to all candidates of this type
        dists = sub.apply(
            lambda r: calculate_distance(s_lat, s_lon, float(r["Lat"]), float(r["Long"])),
            axis=1
        )
        idx = int(np.argmin(dists.values))

        p = sub.iloc[idx]
        p_lat, p_lon = float(p["Lat"]), float(p["Long"])
        d_km = float(dists.iloc[idx])
        color = type_color.get(prox_type, "orange")

        # 1) Marker at the proximity point with a distance label
        folium.Marker(
            location=[p_lat, p_lon],
            popup=f"{prox_type}: {p['name']}",
            icon=folium.Icon(color="white", icon="map-marker", prefix="fa", icon_color=color)
        ).add_to(site_map)

        # Distance text near the proximity point
        folium.Marker(
            location=[p_lat, p_lon],
            icon=DivIcon(
                icon_size=(160, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12px; color:{color};"><b>{s_name} → {prox_type}: {d_km:.2f} km</b></div>',
            ),
        ).add_to(site_map)

        # 2) Line from the site to the proximity
        folium.PolyLine(
            locations=[(s_lat, s_lon), (p_lat, p_lon)],
            color=color,
            weight=3,
            opacity=0.85,
            tooltip=f"{s_name} ↔ {prox_type}: {d_km:.2f} km",
        ).add_to(site_map)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
